In [1]:
!pip install biopython

Import essential libraries

In [2]:
from Bio import SeqIO
from pprint import pprint

(1) Using SeqIO method to count number of records and save the results

In [3]:
counter=0                                                      #counter to keep track of record counts
for seq_record in SeqIO.parse("dna2.fasta", "fasta"):   #going through each record, uisng SeqIO module 
        counter=counter+1                                      #increment counter
print("Number of records: " + str(counter))                    #print number of records info

Number of records: 18


(2) Longest and shortest sequences

In [12]:
def seqLenInfo(fasta_file):    
    seqLen = {}
    for seq_record in SeqIO.parse(fasta_file, "fasta"):           #going through each record
            seqLen[seq_record.id] = len(seq_record.seq[:])        #save id and length info in a dict
                                                             
    print("Partial sequence records of form  <id: length>")
    pprint({k: seqLen[k] for k in list(seqLen)[:5]})              #print top 5 records form the dict  
    print("------------------------------------------------")

    max_value = max(seqLen.values())                              #find the longest sequence from the dict
    max_keys = [k for k, v in seqLen.items() if v == max_value]   #get id for the longest sequence

    print("Longest sequence length: " + str(seqLen[max_keys[0]])) #print longest sequence length

    if len(max_keys)>1:                                           #check if duplicae longest sequence exist
        print("There are more than one longest sequences.")
    else:
        print("There are no duplicate longest sequences.")
    print("------------------------------------------------")

    min_value = min(seqLen.values())                              #find the sequence sequence from the dict
    min_keys = [k for k, v in seqLen.items() if v == min_value]   #get id for the sequence sequence
    
    print("Shortest sequence length: " + str(seqLen[min_keys[0]])) #print shortest sequence length

    if len(min_keys)>1:                                           #check if duplicae shortest sequence exist
        print("There are more than one shortest sequences.")
    else:
        print("There are no duplicate shortest sequences.")
    print("------------------------------------------------")

In [13]:
seqLenInfo("dna2.fasta") #print file info of shortest and longest sequences

Partial sequence records of form  <id: length>
{'gi|142022655|gb|EQ086233.1|255': 4894,
 'gi|142022655|gb|EQ086233.1|304': 1151,
 'gi|142022655|gb|EQ086233.1|396': 4076,
 'gi|142022655|gb|EQ086233.1|45': 3511,
 'gi|142022655|gb|EQ086233.1|91': 4635}
------------------------------------------------
Longest sequence length: 4894
There are no duplicate longest sequences.
------------------------------------------------
Shortest sequence length: 115
There are no duplicate shortest sequences.
------------------------------------------------


(3) Find ORFs

In [4]:
# Find ORF in Sequence
def find_orf(seq_record):      
    start_indices = []
    stop_indices = []
    for i in range(1, len(seq_record)):         #find the start codon (ATG) indices
        if seq_record[i:i+3] == "ATG":          #in the whole given sequence
            start_indices.append(i)
    
    for i in range(1, len(seq_record)):         #find all the stop codon indices
        stops = ["TAA", "TAG", "TGA"]           #in the whole given sequence
        if seq_record[i:i+3] in stops:
            stop_indices.append(i)

    orfs = []
    mark = 0
    for i in range(0,len(start_indices)):       #compare the relative positions of
        for j in range(0, len(stop_indices)):   #the start and stop codons
            if start_indices[i] < stop_indices[j] and start_indices[i] > mark:
                orfs.append(seq_record[start_indices[i]:stop_indices[j]+3])
                mark = stop_indices[j]+3
                break
    return orfs

*3.1 What is the length of the longest ORF in the file?*

*3.2 What is the identifier of the sequence containing the longest ORF?*

In [5]:
longestOrfs = {}                                              #create a dict and store the longest
                                                              #ORFs for the all the sequences in a file
                                                              #with their parent id's
for seq_record in SeqIO.parse("dna2.fasta", "fasta"):  
        longestOrfs[seq_record.id] = max(find_orf (seq_record.seq[:]),key=len)

longest_value = max(longestOrfs.values(),key=len)             #get the longest ORF's length among the others
longest_id = [k for k, v in longestOrfs.items() if v == longest_value] #get its id

print(len(longest_value))                                     #print the longest ORF's length
print(longest_id)                                             #print its respective id

388
['gi|142022655|gb|EQ086233.1|255']


*3.3 For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier?* 

In [17]:
given_id = input("Enter a sequence id (e.g. gi|142022655|gb|EQ086233.1|323): \n")   #prompt the user to enter an id  
id_found = False                                         #flag to check if id is not found

for id in longestOrfs:                                   #scan in the dictionary of the longestOrfs
    if id == given_id:                                   #when id matches with the users input 
        print ("\nThe longest ORF for the given id is\n") 
        print (longestOrfs[given_id])                    #print the longest ORF when input id matches
        print("\nand has a length of " + str(len(longestOrfs[given_id])))
        id_found = True
        
if not id_found:
    print("id not found!")                               #notify the user if id is not found

Enter a sequence id (e.g. gi|142022655|gb|EQ086233.1|323): 
gi|142022655|gb|EQ086233.1|16

The longest ORF for the given id is

ATGGCGCGACGCGGCGGAGTTCATGGTGCTCGGCGCCGGCAGCGTGCAGGTGTGCACCGCCGCGATGCATTACGGATTCCGGATCGTGTCGGACCTGGCCGACGGATTGTCGAACTGGATGGACGAGAAGGGCTACGCGACGCTCGACGACATTCGCGGCCGCGCGGTGCCGAACGTGA

and has a length of 179


*3.4 What is the starting position of the longest ORF in the sequence that contains it?* 

In [16]:
for seq_record in SeqIO.parse("dna2.fasta", "fasta"):  #to check the start position of longest ORF 
                                                              #in the sequence that contains it, find the sequence
           if seq_record.id == longest_id[0]:                 #by matching its id with ids' for the whole file sequences
            ind = seq_record.seq[:].find(longest_value)       #identify where the ORF is in the sequence
            position = ind + 1                                #get the starting position
            print(position)                                   #print the starting position
            break                                             #terminate the searchin assuming no duplicates 


2780


(4) Determining repeats

In [1]:
from collections import Counter                          #import essential libraries
from itertools import product   
from Bio import SeqIO
from pprint import pprint

In [2]:
def find_repeats(n, fasta_file):                         #function that takes a file and finds repeats
    repeatsDict = {}
    for seq_record in SeqIO.parse(fasta_file, "fasta"):  #scan the given file, find repeats stats for each record
        newRepeatsDict = dict(("".join(codon), seq_record.seq[:].upper().count(''.join(codon))) for codon in product("ATCG", repeat=n))    
        repeatsDict = dict(Counter(repeatsDict) + Counter(newRepeatsDict)) #combine new repeats stats with the existing ones 

    repeatsDict = {k:v for k, v in repeatsDict.items() if v != 0} #remove the redundant repeats (does't exist, 0 occuren.)
    return repeatsDict                                   #return the repeats dictionary

In [3]:
'''
while True:                 
    try:
        n=int(input("Enter length of repeats: "))        #prompt the user enter a valid input
        break
    except:                                              #handle exception of type mismatch
        pass
    print("\nIncorrect input, try again")  
'''
repeats = find_repeats (12, "dna2.fasta")                 #call the find_repeats function at the given input
most_repeats =  max(repeats.values())                    #determine the longest of the repeats determined

print("\nPartial repeat records of form  <codon: no. of repeats>") #print a summary of the repeats
pprint({k: repeats[k] for k in list(repeats)[:5]})       #with a mxmimum of 5 repeats in the record
print("------------------------------------------------")
print("Most number of repeats for the given repeat length is:" + str(most_repeats))
sorted(repeats.items(), key=lambda x: x[1], reverse=True)

KeyboardInterrupt: 

# --------------------------------------End of document--------------------------------